In [ ]:
!nvidia-smi

Sat May  7 12:39:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import tweepy
import tweepy as tw
# your Twitter API key and API secret
my_api_key = "zyHUhrv4ityGqHDFWfQk9FKMq"
my_api_secret = "XNs6pzHTpXLoguzuuBB0JIe6ewhC9x1n8zV9WZu5Wk4MQGyWz9"
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
search_query = " -filter:retweets"
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="en",
              since="2022-05-24").items(2500)#tanggal diganti sama tanggal hari tsb
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 2500


In [ ]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,💧Brendan Quinlivan,,,False,2022-05-24 01:21:02,"@MyFirstCousin In this case, sworn at",None,Twitter for iPhone
1,FLATSEVEN_US,,FLATSEVEN_US,False,2022-05-24 01:21:02,FLATSEVEN Mens Slim Fit Dress Shirts with Tie ...,None,twittbot.net
2,Peaches,,My own little corner.,False,2022-05-24 01:21:02,See I’ve experienced niggas telling me they lo...,None,Twitter for iPhone
3,Mittens the Kitten 😼,,I'm Mittens the Kitten!\nI'm here to kick butt...,False,2022-05-24 01:21:02,...like that rock? https://t.co/xjVPGxxpml,None,Twitter for Android
4,syafira.,,"liverpool, red velvet, stephen strange enthusi...",False,2022-05-24 01:21:02,where's the trailer hello @MarvelStudios,None,Twitter for Android


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tweets_df.to_csv('Tweets-2022-05-24.csv')#jgn lupa diganti tanggalnya

Pre-Processing

In [ ]:
import nltk
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
stop_words=set(nltk.corpus.stopwords.words('english'))

In [ ]:
def clean_text(headline):
      le=WordNetLemmatizer()
      word_tokens=word_tokenize(headline)
      tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
      cleaned_text=" ".join(tokens)
      return cleaned_text


In [ ]:
tweets_df['cleaned_text']=tweets_df['text'].apply(clean_text)

TFIDF vectorization on the text column

In [ ]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000)
vect_text=vect.fit_transform(tweets_df['cleaned_text'])

LDA on the vectorized text

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,
learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(vect_text)


Result

In [ ]:
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  4.147353487205024 %
Topic  1 :  4.147354112637864 %
Topic  2 :  4.147368576994597 %
Topic  3 :  4.147359892192386 %
Topic  4 :  4.1473550234904035 %
Topic  5 :  62.67377344197016 %
Topic  6 :  4.147354410769453 %
Topic  7 :  4.147362553444848 %
Topic  8 :  4.1473652693631395 %
Topic  9 :  4.147353231932135 %


In [ ]:
vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
     print("")
     print("Topic "+str(i)+": ")
     for t in sorted_words:
            print(t[0],end=" ")



Topic 0: 
http co done true live free like could funny girl 
Topic 1: 
love http co please issue till moment send song without 
Topic 2: 
back look game http co friend always still real like 
Topic 3: 
best today think help wrong want tweet call find night 
Topic 4: 
morning co http good warning flood neosho tatum video near 
Topic 5: 
need year post welcome school cause http co dude hear 
Topic 6: 
good nice wait know yeah little time called lmao thing 
Topic 7: 
right going birthday happy talking since trying every work follow 
Topic 8: 
great avatar claim adidas_meta_ sure shirt amazing ready point meet 
Topic 9: 
co http like thank would really take never know life 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
